In [1]:
"""
SIFT_CLASSIC.py: Near Real-Time EEG Intervention for Epilepsy and Parkinson's
- Offline Mode: Processes pre-recorded EEG data (CHB-MIT dataset) and generates visualizations.
- Hardware Mode: Streams live EEG data via LSL for real-time intervention (requires pylsl and liblsl).
- Outputs: Plots, logs, and numpy arrays saved to ./output/.
- Dependencies: mne, numpy, scipy, matplotlib, pylsl (for hardware mode).
- Dataset: CHB-MIT (chb01_03.edf for seizure, chb01_01.edf for normal).
- License: GNU General Public License v3.0
"""

import os
import logging
import numpy as np
import mne
import scipy.signal as signal
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

# Try to set an interactive backend for displaying plots, fall back to Agg if unavailable
try:
    import tkinter
    plt.switch_backend('TkAgg')  # Interactive backend for PyCharm
    interactive = True
except ImportError:
    print("Warning: tkinter not available, falling back to non-interactive plotting (Agg backend).")
    plt.switch_backend('Agg')  # Non-interactive backend for saving plots
    interactive = False

# Try to import pylsl for hardware mode
try:
    from pylsl import StreamInlet, resolve_stream, StreamOutlet, StreamInfo
    pylsl_available = True
except ImportError as e:
    print(f"Warning: pylsl library not available. Hardware mode will be disabled.")
    print(f"To enable hardware mode, install liblsl (e.g., `conda install -c conda-forge liblsl`) and pylsl (`pip install pylsl`).")
    print(f"Error details: {e}")
    pylsl_available = False

# Directory setup (absolute path for local runs)
eeg_data_dir = '/mnt/datatank/eeg_data/'  # Use your clinical data directory
output_dir = './output/'
os.makedirs(output_dir, exist_ok=True)

# EDF file paths
seizure_file = eeg_data_dir + 'chb01_03.edf'
normal_file = eeg_data_dir + 'chb01_01.edf'

# Setup logging
logging.basicConfig(filename=os.path.join(output_dir, 'sift_interventions_epilepsy.log'), level=logging.INFO)
logging.basicConfig(filename=os.path.join(output_dir, 'sift_interventions_parkinsons.log'), level=logging.INFO)

# SIFT parameters
fs = 256  # Sampling frequency (Hz)
seizure_start = 2996.0  # Seizure onset time for chb01_03.edf (seconds)
channel = 'CZ-PZ'  # Channel to analyze
mode = 'offline'  # 'offline' or 'hardware'

# Load EEG data
try:
    raw_seizure = mne.io.read_raw_edf(seizure_file, preload=True)
    raw_normal = mne.io.read_raw_edf(normal_file, preload=True)
except FileNotFoundError as e:
    raise FileNotFoundError(f"Could not find {e.filename}. Ensure the CHB-MIT dataset is in {eeg_data_dir}.")

# Get channel data
channels = raw_seizure.ch_names
print(f"Available channels: {channels}")
eeg_seizure = raw_seizure.get_data(picks=channel)[0] * 1e6  # Convert to μV
eeg_normal = raw_normal.get_data(picks=channel)[0] * 1e6  # Convert to μV
print("eeg_seizure shape:", eeg_seizure.shape)
print("eeg_seizure min/max:", eeg_seizure.min(), eeg_seizure.max())
t = np.arange(len(eeg_seizure)) / fs  # Time vector

# Simulate Parkinson's by adding a 20 Hz beta signal to normal EEG
beta_freq = 20  # Hz
beta_amplitude = 50  # μV
beta_signal = beta_amplitude * np.sin(2 * np.pi * beta_freq * t)
eeg_parkinsons = eeg_normal + beta_signal

# SIFT processing
def sift_processing(eeg_signal, fs, mode='epilepsy'):
    # Bandpass filter for alpha (8-13 Hz)
    alpha_low, alpha_high = 8, 13
    b_alpha, a_alpha = signal.butter(4, [alpha_low / (fs / 2), alpha_high / (fs / 2)], btype='band')
    alpha_signal = signal.filtfilt(b_alpha, a_alpha, eeg_signal)

    # Bandpass filter for beta (13-30 Hz) for Parkinson's
    beta_low, beta_high = 13, 30
    b_beta, a_beta = signal.butter(4, [beta_low / (fs / 2), beta_high / (fs / 2)], btype='band')
    beta_signal = signal.filtfilt(b_beta, a_beta, eeg_signal)

    # Detect spikes (for epilepsy) or excessive beta (for Parkinson's)
    spike_threshold = 100  # μV
    beta_threshold = 30  # μV
    spike_count = 0
    beta_count = 0
    modified_signal = eeg_signal.copy()

    for i in range(len(eeg_signal)):
        if mode == 'epilepsy' and abs(eeg_signal[i]) > spike_threshold:
            modified_signal[i] = alpha_signal[i]  # Replace spike with alpha
            spike_count += 1
        elif mode == 'parkinsons' and abs(beta_signal[i]) > beta_threshold:
            modified_signal[i] = alpha_signal[i]  # Suppress beta with alpha
            beta_count += 1

    return modified_signal, spike_count, beta_count

# Process EEG data
if mode == 'offline':
    # Process epilepsy data
    eeg_signal_modified_epilepsy, spike_count, _ = sift_processing(eeg_seizure, fs, mode='epilepsy')
    logging.info(f"Epilepsy interventions: {spike_count} spike suppressions, {len(eeg_seizure) - spike_count} alpha enhancements")
    print(f"Epilepsy interventions: {spike_count} spike suppressions, {len(eeg_seizure) - spike_count} alpha enhancements")

    # Process Parkinson's data
    eeg_signal_modified_parkinsons, _, beta_count = sift_processing(eeg_parkinsons, fs, mode='parkinsons')
    logging.info(f"Parkinson's interventions: {beta_count} beta suppressions, {len(eeg_parkinsons) - beta_count} alpha enhancements")
    print(f"Parkinson's interventions: {beta_count} beta suppressions, {len(eeg_parkinsons) - beta_count} alpha enhancements")

    # Plot results - Epilepsy (near seizure onset for visibility)
    start_idx = int(2996 * fs)  # Start at 2996 seconds (seizure onset)
    end_idx = start_idx + int(2 * fs)  # 2 seconds window
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(t[start_idx:end_idx], eeg_normal[start_idx:end_idx], label='Normal EEG', linestyle='--', color='green', alpha=0.7, linewidth=1.5)
    ax.plot(t[start_idx:end_idx], eeg_seizure[start_idx:end_idx], label='Original EEG', color='blue', alpha=0.7, linewidth=1.5)
    ax.plot(t[start_idx:end_idx], eeg_signal_modified_epilepsy[start_idx:end_idx], label='Modified EEG', color='orange', alpha=0.9, linewidth=2)  # Plot last, make thicker
    ax.set_ylim(-150, 150)
    ax.set_xlim(2996, 2998)  # Adjust x-axis to show the seizure onset range
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Amplitude (μV)')
    ax.set_title('SIFT EEG Intervention - Epilepsy (Near Seizure Onset)')
    ax.legend()
    ax.grid(True)  # Add grid for better readability

    # Add zoomed-in inset to highlight a spike suppression
    # Find a spike in the plotted range
    spike_indices = np.where(np.abs(eeg_seizure[start_idx:end_idx]) > 100)[0]
    if len(spike_indices) > 0:
        spike_idx = spike_indices[0] + start_idx  # First spike in the range
        zoom_start = max(spike_idx - int(0.1 * fs), start_idx)  # 0.1s before spike
        zoom_end = min(spike_idx + int(0.1 * fs), end_idx)  # 0.1s after spike
        ax_inset = inset_axes(ax, width="30%", height="30%", loc='upper left')
        ax_inset.plot(t[zoom_start:zoom_end], eeg_seizure[zoom_start:zoom_end], color='blue', alpha=0.7, linewidth=1.5)
        ax_inset.plot(t[zoom_start:zoom_end], eeg_signal_modified_epilepsy[zoom_start:zoom_end], color='orange', alpha=0.9, linewidth=2)
        ax_inset.set_ylim(-150, 150)
        ax_inset.grid(True)

    plt.savefig(os.path.join(output_dir, 'sift_eeg_results_epilepsy.png'))
    if interactive:
        plt.show()
    plt.close()

    # Plot results - Parkinson's
    plt.figure(figsize=(10, 6))
    plt.plot(t[:int(2 * fs)], eeg_normal[:int(2 * fs)], label='Normal EEG', linestyle='--', color='green', alpha=0.7, linewidth=1.5)
    plt.plot(t[:int(2 * fs)], eeg_parkinsons[:int(2 * fs)], label='Original EEG', color='blue', alpha=0.7, linewidth=1.5)
    plt.plot(t[:int(2 * fs)], eeg_signal_modified_parkinsons[:int(2 * fs)], label='Modified EEG', color='orange', alpha=0.9, linewidth=2)
    plt.ylim(-150, 150)
    plt.xlim(0, 2)
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude (μV)')
    plt.title('SIFT EEG Intervention - Parkinson\'s')
    plt.legend()
    plt.grid(True)  # Add grid for better readability
    plt.savefig(os.path.join(output_dir, 'sift_eeg_results_parkinsons.png'))
    if interactive:
        plt.show()
    plt.close()

    # Save processed data
    np.save(os.path.join(output_dir, 'eeg_signal_original_epilepsy.npy'), eeg_seizure)
    np.save(os.path.join(output_dir, 'eeg_signal_modified_epilepsy.npy'), eeg_signal_modified_epilepsy)
    np.save(os.path.join(output_dir, 'eeg_signal_original_parkinsons.npy'), eeg_parkinsons)
    np.save(os.path.join(output_dir, 'eeg_signal_modified_parkinsons.npy'), eeg_signal_modified_parkinsons)

    print(f"SIFT simulation complete. Results saved to {output_dir}")

elif mode == 'hardware' and pylsl_available:
    # Hardware mode: Stream EEG data via LSL
    streams = resolve_stream('type', 'EEG')
    inlet = StreamInlet(streams[0])
    outlet = StreamOutlet(StreamInfo('SIFT_Control', 'Markers', 1, 0, 'int32', 'sift_control'))

    spike_count = 0
    beta_count = 0
    chunk_size = int(fs * 0.05)  # 50ms chunks

    while True:
        chunk, timestamps = inlet.pull_chunk(timeout=1.0, max_samples=chunk_size)
        if not chunk:
            continue

        chunk = np.array(chunk)[:, 0] * 1e6  # Convert to μV
        modified_chunk, spikes, betas = sift_processing(chunk, fs, mode='epilepsy')
        spike_count += spikes
        beta_count += betas

        # Send control signals via LSL
        if spikes > 0:
            outlet.push_sample([1])  # Spike suppression marker
        elif betas > 0:
            outlet.push_sample([2])  # Beta suppression marker

        logging.info(f"Real-time interventions: {spike_count} spike suppressions, {beta_count} beta suppressions")
else:
    raise ValueError("Hardware mode selected but pylsl is not available. Install pylsl or use offline mode.")

To enable hardware mode, install liblsl (e.g., `conda install -c conda-forge liblsl`) and pylsl (`pip install pylsl`).
Error details: cannot import name 'resolve_stream' from 'pylsl' (/home/wayne/jupyter_env/lib/python3.12/site-packages/pylsl/__init__.py)
Extracting EDF parameters from /mnt/datatank/eeg_data/chb01_03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


/ssd/tmp/ipykernel_68344/2452068002.py:60: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw_seizure = mne.io.read_raw_edf(seizure_file, preload=True)


Reading 0 ... 921599  =      0.000 ...  3599.996 secs...
Extracting EDF parameters from /mnt/datatank/eeg_data/chb01_01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...


/ssd/tmp/ipykernel_68344/2452068002.py:61: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw_normal = mne.io.read_raw_edf(normal_file, preload=True)


Available channels: ['FP1-F7', 'F7-T7', 'T7-P7', 'P7-O1', 'FP1-F3', 'F3-C3', 'C3-P3', 'P3-O1', 'FP2-F4', 'F4-C4', 'C4-P4', 'P4-O2', 'FP2-F8', 'F8-T8', 'T8-P8-0', 'P8-O2', 'FZ-CZ', 'CZ-PZ', 'P7-T7', 'T7-FT9', 'FT9-FT10', 'FT10-T8', 'T8-P8-1']
eeg_seizure shape: (921600,)
eeg_seizure min/max: -491.72161172161174 469.45054945054943
Epilepsy interventions: 53655 spike suppressions, 867945 alpha enhancements
Parkinson's interventions: 540727 beta suppressions, 380873 alpha enhancements
SIFT simulation complete. Results saved to ./output/
